In [ ]:
response = graph.invoke("What is 3 multiplied by 5, then add 10?")
print(response.content)

ValueError: No message found in input

In [25]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict
from typing import Annotated
from langgraph.graph import START, StateGraph, END
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage

# Load API keys
load_dotenv()
openrouter_key = os.getenv("OPENROUTER_API_KEY")
openrouter_base_url = os.getenv("OPENAI_BASE_URL")

# LLM setup
llm = ChatOpenAI(
    model_name="deepseek/deepseek-chat-v3.1:free",
    api_key=openrouter_key,
    base_url=openrouter_base_url
)

# ---------- State ----------
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ---------- Tools ----------
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

@tool
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

# Bind tools
llm_with_tools = llm.bind_tools([add, multiply])

# ---------- Nodes ----------
def call_llm(state: State) -> State:
    result = llm_with_tools.invoke(state["messages"])
    return {"messages": [result]}

tool_node = ToolNode([add, multiply])

# ---------- Graph ----------
# ---------- Graph ----------
builder = StateGraph(State)
builder.add_node("llm", call_llm)
builder.add_node("tools", tool_node)

builder.add_edge(START, "llm")

# Conditional edge: if the LLM calls a tool, go to "tools"
builder.add_conditional_edges(
    "llm",
    lambda state: "tools" if state["messages"][-1].tool_calls else END,
    {"tools": "tools", END: END},
)

# After a tool executes, always go back to llm
builder.add_edge("tools", "llm")

graph = builder.compile()


# ---------- Run ----------
response = graph.invoke({
    "messages": [HumanMessage(content="What is 3 multiplied by 5, then add 10?")]
})

print(response["messages"][-1].content)


3 multiplied by 5 is 15, and then adding 10 gives you 25.


In [27]:
import os
from dotenv import load_dotenv
from typing_extensions import TypedDict
from typing import Annotated, List

from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, tool
from langchain.memory import ConversationBufferMemory
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

# ---------- Load keys ----------
load_dotenv()
openrouter_key = os.getenv("OPENROUTER_API_KEY")
openrouter_base_url = os.getenv("OPENAI_BASE_URL")

llm = ChatOpenAI(
    model_name="deepseek/deepseek-chat-v3.1:free",
    api_key=openrouter_key,
    base_url=openrouter_base_url
)

# ---------- State ----------
class State(TypedDict):
    messages: Annotated[list, add_messages]
    chat_history: List

# ---------- Tools ----------
@tool
def add_numbers(inputs: str) -> str:
    """Add two integers. Input format: 'x y'"""
    x, y = map(int, inputs.split())
    return str(x + y)

@tool
def multiply_numbers(inputs: str) -> str:
    """Multiply two integers. Input format: 'x y'"""
    x, y = map(int, inputs.split())
    return str(x * y)

tools = [add_numbers, multiply_numbers]

# ---------- Agent ----------
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent_with_tool = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

# ---------- Graph Node ----------
def tool_agent(state: State) -> State:
    res = agent_with_tool.invoke({
        "input": state["messages"][-1].content,
        "chat_history": state.get("chat_history", [])
    })
    # Append new message to state
    return {
        "messages": state["messages"] + [res["output"]],
        "chat_history": state.get("chat_history", []) + [res]
    }

# ---------- Graph ----------
workflow = StateGraph(State)
workflow.add_node("tool_agent", tool_agent)
workflow.add_edge(START, "tool_agent")
workflow.add_edge("tool_agent", END)

graph = workflow.compile()

# ---------- Run ----------
response = graph.invoke({
    "messages": [{"type": "human", "content": "multiply 12 and 8 then add 10"}],
    "chat_history": []
})

print(response)





> Entering new AgentExecutor chain...
```json
{
    "action": "multiply_numbers",
    "action_input": "12 8"
}
```
Observation: 96
Thought:```json
{
    "action": "add_numbers",
    "action_input": "96 10"
}
```
Observation: 106
Thought:

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit exceeded: free-models-per-day. Add 10 credits to unlock 1000 free model requests per day', 'code': 429, 'metadata': {'headers': {'X-RateLimit-Limit': '50', 'X-RateLimit-Remaining': '0', 'X-RateLimit-Reset': '1757980800000'}, 'provider_name': None}}, 'user_id': 'user_325zr9riwtnIO3ksq81BGD3LWs8'}